In [ ]:
# final

# -*- coding: utf-8 -*-
"""
Mapa completo — Folium/Leaflet (geolocalização + PT-BR no Medir + legenda à direita)
- Legenda no canto inferior direito (largura maior e até 70% da tela) com Mostrar/Ocultar.
- MiniMap no canto inferior esquerdo.
- Telefones das administradoras nos popups das LINHAS e também dos MARCOS KM.
- Saudação embutida: aparece ao ENTRAR na área da Cia (geofence), com município, rodovia e concessionária do marco mais próximo (raio 1 km).
- Botão ℹ️: se DENTRO da área, mostra SP/KM/Município/Concessionária; se FORA, mostra apenas Rodovia (SP), KM e Município.

Requisitos:
    pip install pandas folium shapely
Arquivos na mesma pasta:
    - Malha.xlsx
    - coordenadas.xlsx
    - municipios_selecionados.xlsx
    - bases.xlsx
Saída:
    - mapa_completo_final.html
"""

import json
import re
from pathlib import Path
from collections import defaultdict

import folium
from folium.plugins import (
    LocateControl, MarkerCluster, Fullscreen, MiniMap, MeasureControl
)
from folium.features import GeoJsonPopup
from shapely import wkt
import pandas as pd
import numpy as np
from branca.element import Element

# =========================
# Configurações de caminhos
# =========================
BASE = Path(".")
ARQ_KM = BASE / "coordenadas.xlsx"
ARQ_BASES = BASE / "bases.xlsx"
ARQ_MUN = BASE / "municipios_selecionados.xlsx"
ARQ_MALHA = BASE / "Malha.xlsx"
SAIDA_HTML = BASE / "mapa_completo_final.html"

# =========================
# Helpers de normalização
# =========================
def norm_rod_str(s: str) -> str:
    s = str(s).upper().strip()
    s = re.sub(r"\s+", " ", s)
    s = re.sub(r"^SPA\b", "SP", s)
    return s

def normalize_concessionaria(name: str) -> str:
    n = (name or "").strip().upper()
    if "EIXO" in n or "PIPA" in n: return "Concessionária EIXOSP - PIPA"
    if "ECOVIAS" in n: return "Concessionária Ecovias Noroeste Paulista"
    if "ENTREVIAS" in n: return "Concessionária Entrevias"
    if "INTERVIAS" in n: return "Concessionária Intervias"
    if "VIA PAULISTA" in n or "VIAPAULISTA" in n: return "Concessionária ViaPaulista"
    return f"Concessionária {name.strip()}" if name else "Concessionária"

def padroniza_admin_row(row) -> str:
    admin = str(row["ADMINISTRAÇÃO"]).upper().strip()
    cons = str(row["CONSERVAÇÃO"]).strip()
    sp_str = str(row["SP"]).strip()
    if "DER" in admin:
        m = re.search(r"(\d{2})", cons)
        if m: return f"DER {m.group(1)}"
        m2 = re.search(r"DER\D*(\d{2})", admin)
        if m2: return f"DER {m2.group(1)}"
        if sp_str in ("149/215", "348/310"): return "DER 04"
        return "DER 04"
    if "CONCESSION" in admin or "CONCESSIONÁRIA" in admin:
        return normalize_concessionaria(cons)
    if sp_str in ("149/215", "348/310"): return "DER 04"
    m = re.search(r"(\d{2})", cons)
    if m: return f"DER {m.group(1)}"
    return "DER 04"

def shapely_to_geojson_feature(r, cor, pelotao):
    popup_html = (
        f"<b>Município:</b> {r['name_muni']}<br>"
        f"<b>Pelotão:</b> {pelotao or '-'}<br>"
        f"<b>Batalhão:</b> {r.get('BTLH','-')}<br>"
        f"<b>CPI:</b> {r.get('CPI','-')}"
    )
    return {
        "type": "Feature",
        "properties": {"popup_html": popup_html},
        "geometry": r["geometry"].__geo_interface__,
    }

def geom_to_rings_latlng(geom):
    polys = []
    if geom.geom_type == "Polygon":
        ext = [[lat, lng] for (lng, lat) in list(geom.exterior.coords)]
        holes = [[[lat, lng] for (lng, lat) in list(inter.coords)] for inter in geom.interiors]
        polys.append([ext] + holes)
    elif geom.geom_type == "MultiPolygon":
        for p in geom.geoms:
            ext = [[lat, lng] for (lng, lat) in list(p.exterior.coords)]
            holes = [[[lat, lng] for (lng, lat) in list(inter.coords)] for inter in p.interiors]
            polys.append([ext] + holes)
    return polys

# =========================
# Pipeline principal
# =========================
def main():
    # ---------- Leitura ----------
    df_km = pd.read_excel(ARQ_KM)
    df_bases = pd.read_excel(ARQ_BASES)
    df_municipios = pd.read_excel(ARQ_MUN)
    df_malha = pd.read_excel(ARQ_MALHA, header=None, skiprows=1)

    df_malha.columns = [
        "Tipo","SP","KM INICIAL","KM FINAL","EXTENSÃO","MUNICÍPIO","RODOVIA","JURISDIÇÃO",
        "ADMINISTRAÇÃO","CONSERVAÇÃO","SUPERFÍCIE","PELOTÃO","Sentido","CPI","BTLH","CIA","PEL/GP"
    ]
    for c in ["KM INICIAL","KM FINAL"]:
        df_malha[c] = pd.to_numeric(df_malha[c], errors="coerce")

    # ---------- Normalizações ----------
    df_malha["ADMIN_FINAL"] = df_malha.apply(padroniza_admin_row, axis=1)
    df_malha["COD_ROD"] = (df_malha["Tipo"].astype(str) + " " + df_malha["SP"].astype(str)).apply(norm_rod_str)

    df_municipios["geometry"] = df_municipios["geometry"].apply(wkt.loads)

    info = (
        df_malha[["MUNICÍPIO","PELOTÃO","BTLH","CPI"]]
        .dropna(subset=["MUNICÍPIO"]).drop_duplicates("MUNICÍPIO")
    )
    mapeia_pel = {1:"1º Pel",2:"2º Pel",3:"3º Pel"}
    def map_pel(v):
        try:
            if pd.isna(v): return None
            return mapeia_pel.get(int(v))
        except Exception:
            return None
    info["PELOTÃO"] = info["PELOTÃO"].apply(map_pel)
    map_info = info.set_index("MUNICÍPIO").to_dict("index")

    df_municipios["PELOTÃO"] = df_municipios["name_muni"].map(lambda x: map_info.get(x,{}).get("PELOTÃO"))
    df_municipios["BTLH"] = df_municipios["name_muni"].map(lambda x: map_info.get(x,{}).get("BTLH"))
    df_municipios["CPI"] = df_municipios["name_muni"].map(lambda x: map_info.get(x,{}).get("CPI"))

    df_km["_ROD"] = df_km["rodovia"].apply(norm_rod_str)

    def get_info(row):
        mask = (
            (df_malha["COD_ROD"] == row["_ROD"]) &
            (df_malha["KM INICIAL"] <= float(row["km"])) &
            (df_malha["KM FINAL"] >= float(row["km"]))
        )
        t = df_malha.loc[mask]
        if not t.empty:
            t0 = t.iloc[0]
            return pd.Series([t0["MUNICÍPIO"], f"{t0['Tipo']} {t0['SP']} - {t0['RODOVIA']}", t0["ADMIN_FINAL"]])
        t2 = df_malha[df_malha["COD_ROD"] == row["_ROD"]]
        if not t2.empty:
            t0 = t2.iloc[0]
            return pd.Series([t0["MUNICÍPIO"], f"{t0['Tipo']} {t0['SP']} - {t0['RODOVIA']}", t0["ADMIN_FINAL"]])
        return pd.Series(["(indef.)", row["_ROD"], "DER 04"])

    df_km[["municipio","rodovia_nome","concessionaria"]] = df_km.apply(get_info, axis=1)

    # ---------- Telefones ----------
    phones = {
        "Concessionária EIXOSP - PIPA": "0800 170 8998",
        "Concessionária Ecovias Noroeste Paulista": "0800 326 3663",
        "Concessionária Entrevias": "0800 3000 333",
        "Concessionária Intervias": "0800 707 1414",
        "Concessionária ViaPaulista": "0800 001 1255",
        "DER 14": "0800 55 55 10",
        "DER 04": "0800 55 55 10",
    }
    def tel_pair(nome):
        tel = phones.get(str(nome), "-")
        digits = re.sub(r"\D+", "", tel) if tel != "-" else ""
        return tel, digits

    # ---------- Mapa base ----------
    mapa = folium.Map(location=[-21.5, -48.5], zoom_start=8,
                      control_scale=True, width="100%", height="100%")

    # Controles
    LocateControl(position="topleft", auto_start=False, keepCurrentZoomLevel=True, flyTo=True,
                  strings={"title":"Mostrar minha localização","popup":"Você está aqui","outsideMapBoundsMsg":"Você está fora da área do mapa"},
                  locateOptions={"enableHighAccuracy":True,"maximumAge":15000,"timeout":10000}).add_to(mapa)
    Fullscreen(position="topleft").add_to(mapa)
    MeasureControl(position="topleft",
                   primary_length_unit='kilometers', secondary_length_unit='meters',
                   primary_area_unit='hectares', secondary_area_unit='sqmeters').add_to(mapa)
    MiniMap(position="bottomleft", toggle_display=True).add_to(mapa)

    # ---------- Municípios ----------
    cores_pelotao = {"1º Pel":"blue","2º Pel":"green","3º Pel":"gray"}
    geojson_names = []
    for pelotao, sub in df_municipios.groupby("PELOTÃO"):
        fg = folium.FeatureGroup(name=f"{pelotao}", show=True)
        for _, r in sub.iterrows():
            cor = cores_pelotao.get(pelotao, "#CCCCCC")
            feat = shapely_to_geojson_feature(r, cor, pelotao)
            g = folium.GeoJson(
                data=feat, name=f"muni_{r['name_muni']}",
                style_function=lambda f, cor=cor: {"fillColor":cor,"color":"black","weight":1,"fillOpacity":0.35},
                highlight_function=lambda f: {"weight":3,"fillOpacity":0.6},
            )
            g.add_child(GeoJsonPopup(fields=["popup_html"], labels=False, parse_html=True, localize=True, max_width=320))
            g.add_to(fg); geojson_names.append(g.get_name())
        fg.add_to(mapa)

    # ---------- Marcos km (com telefone no popup) ----------
    counts = df_km.groupby(["x","y"]).size()
    delta = 0.00005
    offsets = {coord:list(np.linspace(-delta*(cnt-1)/2, delta*(cnt-1)/2, cnt))
               for coord, cnt in counts.items() if cnt > 1}
    used = defaultdict(int)
    cluster = MarkerCluster(name="Marcos Quilométricos").add_to(mapa)
    for _, r in df_km.iterrows():
        coord = (r["x"], r["y"])
        idx = used[coord]
        off = offsets.get(coord, [0])[idx] if coord in offsets else 0
        used[coord] += 1
        tel, tel_digits = tel_pair(r["concessionaria"])
        tel_html = f"<a href='tel:{tel_digits}'>{tel}</a>" if tel_digits else "-"
        folium.Marker(
            location=[r["y"] + off, r["x"]],
            icon=folium.DivIcon(html=f'<div class="km-label" style="font-size:9pt;font-weight:bold;color:black;">{int(r["km"])}</div>'),
            popup=folium.Popup(
                f"<b>Município:</b> {r['municipio']}<br>"
                f"<b>Rodovia:</b> {r['rodovia_nome']}<br>"
                f"<b>Administração:</b> {r['concessionaria']}<br>"
                f"<b>Telefone:</b> {tel_html}",
                max_width=340
            )
        ).add_to(cluster)

    # ---------- Linhas por Administração + telefone ----------
    cores_concessionarias = {
        "Concessionária EIXOSP - PIPA":"red",
        "Concessionária Ecovias Noroeste Paulista":"blue",
        "Concessionária Entrevias":"green",
        "Concessionária Intervias":"orange",
        "Concessionária ViaPaulista":"purple",
        "DER 14":"white", "DER 04":"gray",
    }
    for concess, grp in df_km.groupby("concessionaria"):
        fg = folium.FeatureGroup(name=f"Administração: {concess}", show=False)
        cor = cores_concessionarias.get(concess, "#666666")
        tel, tel_digits = tel_pair(concess)
        tel_html = f"<a href='tel:{tel_digits}'>{tel}</a>" if tel_digits else "-"
        for _, sub in grp.groupby("_ROD"):
            coords = sub.sort_values("km")[["y","x"]].values.tolist()
            folium.PolyLine(
                locations=coords, color=cor, weight=5, opacity=0.85,
                popup=folium.Popup(
                    f"<b>Administração:</b> {concess}<br>"
                    f"<b>Rodovia:</b> {sub['_ROD'].iloc[0]}<br>"
                    f"<b>Telefone:</b> {tel_html}",
                    max_width=340
                )
            ).add_to(fg)
        fg.add_to(mapa)

    # ---------- CSS + Legenda ----------
    style_css = """
<style>
#legend-mapa {
  position: fixed;
  bottom: 16px;
  right: 16px;
  width: 380px;
  max-height: 70vh;
  overflow: auto;
  background: white;
  z-index: 700;
  font-size: 14px;
  border: 2px solid #888;
  border-radius: 10px;
  box-shadow: 0 2px 12px rgba(0,0,0,.18);
  padding: 12px;
  line-height: 1.28;
}
#legend-mapa .legend-header{
  display:flex;align-items:center;justify-content:space-between;
  font-weight:700;margin-bottom:8px;
}
#legend-mapa button.legend-toggle{
  border:1px solid #bbb;border-radius:6px;background:#f6f6f6;
  padding:2px 8px;cursor:pointer;
}
#legend-mapa .sym{width:12px;height:12px;display:inline-block;margin-right:6px;border:1px solid #000;}
@media (max-width: 640px){
  #legend-mapa{ width: 300px; max-height: 60vh; }
}
#geoloc-warn{
  position:fixed;left:50%;transform:translateX(-50%);bottom:12px;
  background:#fff3cd;border:1px solid #ffeeba;padding:8px 12px;border-radius:8px;
  font-size:13px;z-index:1000;display:none;box-shadow:0 2px 8px rgba(0,0,0,.12);
}
</style>
"""
    mapa.get_root().html.add_child(Element(style_css))

    def tel_link(num: str) -> str:
        digits = re.sub(r"\D+", "", num or "")
        return f'<a href="tel:{digits}" style="text-decoration:none;color:#333;">{num}</a>' if digits else "-"

    legenda_body = '<b>Pelotões</b><br>'
    for nm, co in {"1º Pel":"blue","2º Pel":"green","3º Pel":"gray"}.items():
        legenda_body += f'<div style="margin:3px 0;"><i class="sym" style="background:{co};"></i> {nm}</div>'
    legenda_body += '<hr style="margin:8px 0;"><b>Concessionárias / DER</b><br>'
    for nm, co in cores_concessionarias.items():
        numero = phones.get(nm, "-")
        legenda_body += (
            f'<div style="margin:4px 0;">'
            f'<i class="sym" style="background:{co};"></i> {nm}<br>'
            f'<span style="margin-left:20px;font-size:12px;color:#333;">{tel_link(numero)}</span>'
            f'</div>'
        )

    legenda_html = f"""
<div id="legend-mapa">
  <div class="legend-header">
    Legenda
    <button class="legend-toggle"
      onclick="(function(btn){{var c=btn.closest('#legend-mapa');var b=c.querySelector('.legend-body');var sh=b.style.display!=='none';b.style.display=sh?'none':'block';btn.textContent=sh?'Mostrar':'Ocultar';}})(this)">Ocultar</button>
  </div>
  <div class="legend-body">
    {legenda_body}
  </div>
</div>
<div id="geoloc-warn">Para ativar geolocalização, abra o mapa via <b>http://localhost</b> (navegadores bloqueiam GPS em <code>file://</code>).</div>
"""
    mapa.get_root().html.add_child(Element(legenda_html))

    # ---------- Layer control ----------
    folium.LayerControl(collapsed=True).add_to(mapa)

    # ---------- JS: rótulos de KM apenas com zoom >= 12 ----------
    map_var = mapa.get_name()
    mapa.get_root().html.add_child(Element(f"""
<script>
(function() {{
  var map = {map_var};
  function toggleKmLabels() {{
    var show = map.getZoom() >= 12;
    document.querySelectorAll('.km-label').forEach(function(el) {{
      el.style.display = show ? 'block' : 'none';
    }});
  }}
  map.on('zoomend', toggleKmLabels);
  toggleKmLabels();
}})();
</script>
"""))

    # ---------- JS: Destaque e popup nos municípios ----------
    geo_list = ",".join(geojson_names)
    mapa.get_root().html.add_child(Element(f"""
<script>
(function() {{
   var map = {map_var};
   function baseStyle(layer) {{ return {{ color: layer.options.color||'black', weight: 1, fillOpacity: 0.35, fillColor: layer.options.fillColor||'#ccc' }}; }}
   function highlightStyle(layer) {{ var s=baseStyle(layer); s.weight=3; s.fillOpacity=0.6; return s; }}
   function reset(layer) {{ layer.setStyle(layer._baseStyle||baseStyle(layer)); }}
   function highlight(layer) {{ layer.setStyle(highlightStyle(layer)); }}
   function attach(layer) {{
      layer._baseStyle = baseStyle(layer);
      var handler = function(e) {{
         if (window._selectedMunicipio && window._selectedMunicipio !== layer) {{ reset(window._selectedMunicipio); }}
         highlight(layer); window._selectedMunicipio = layer;
         if (layer.getPopup && layer.getPopup()) {{ layer.openPopup(e && e.latlng ? e.latlng : undefined); }}
         if (e && e.originalEvent && e.originalEvent.preventDefault) e.originalEvent.preventDefault();
      }};
      layer.on('click', handler);
      layer.on('touchstart', handler);
      layer.on('mouseover', function() {{ if (window._selectedMunicipio !== layer) highlight(layer); }});
      layer.on('mouseout', function() {{ if (window._selectedMunicipio !== layer) reset(layer); }});
   }}
   var layers = [{geo_list}];
   layers.forEach(function(g) {{
      if (!g) return;
      g.eachLayer(function(l) {{
         if (l && l.eachLayer) {{
            l.eachLayer(function(poly) {{ if (poly instanceof L.Polygon || poly instanceof L.MultiPolygon) attach(poly); }});
         }} else if (l instanceof L.Polygon || l instanceof L.MultiPolygon) {{ attach(l); }}
      }});
   }});
   map.on('click', function() {{ if (window._selectedMunicipio) {{ reset(window._selectedMunicipio); window._selectedMunicipio=null; }} }});
}})();
</script>
"""))

    # ---------- JS: Geofencing + aviso file:// ----------
    geofence_polys = []
    for _, r in df_municipios.iterrows():
        geofence_polys.extend(geom_to_rings_latlng(r["geometry"]))
    mapa.get_root().html.add_child(Element(f"""
<script>
window.__MAPA_GEOFENCE = {json.dumps(geofence_polys)};
(function() {{ if (location.protocol === 'file:') {{ var el=document.getElementById('geoloc-warn'); if(el) el.style.display='block'; }} }})();
function __pip_in_ring(point, ring) {{
  var x = point[1], y = point[0], inside = false;
  for (var i=0, j=ring.length-1; i<ring.length; j=i++) {{
    var xi=ring[i][1], yi=ring[i][0], xj=ring[j][1], yj=ring[j][0];
    var intersect = ((yi>y)!=(yj>y)) && (x < (xj-xi)*(y-yi)/((yj-yi)||1e-12) + xi);
    if (intersect) inside = !inside;
  }}
  return inside;
}}
function __pip_in_poly(point, poly) {{
  if (!poly || !poly.length) return false;
  var exterior = poly[0];
  if (!__pip_in_ring(point, exterior)) return false;
  for (var h=1; h<poly.length; h++) if (__pip_in_ring(point, poly[h])) return false;
  return true;
}}
window.__MAPA_GEOFENCE_OK = function(latlng) {{
  if (!window.__MAPA_GEOFENCE || !window.__MAPA_GEOFENCE.length) return true;
  var p = [latlng[0], latlng[1]];
  for (var i=0;i<window.__MAPA_GEOFENCE.length;i++) if (__pip_in_poly(p, window.__MAPA_GEOFENCE[i])) return true;
  return false;
}};
</script>
"""))

    # ---------- JS: Saudação EMBUTIDA (raio 1 km) ----------
    SAUD_JS = r"""
// saudacao_localizacao.js — embutido (raio 1 km)
const CIA_LABEL = "1ª Cia do 3º BPRv";
const DISTANCIA_MAX_KM = 1.0;
const MOSTRAR_APENAS_AO_ENTRAR = true;

function calcularDistancia(lat1, lon1, lat2, lon2) {
  const R = 6371;
  const dLat = (lat2 - lat1) * Math.PI / 180;
  const dLon = (lon2 - lon1) * Math.PI / 180;
  const a = Math.sin(dLat/2)**2 +
            Math.cos(lat1*Math.PI/180) * Math.cos(lat2*Math.PI/180) *
            Math.sin(dLon/2)**2;
  const c = 2 * Math.atan2(Math.sqrt(a), Math.sqrt(1-a));
  return R * c;
}
function obterMapaFolium() {
  if (window.map && typeof window.map.setView === "function") return window.map;
  for (const k in window) {
    try { if (k.startsWith("map_") && window[k] && typeof window[k].setView === "function") return window[k]; } catch (_) {}
  }
  return null;
}
function extrairInfoPopup(html) {
  if (!html) return { municipio: "—", rodovia: "—", adm: "—" };
  const mMun = html.match(/Munic[ií]pio:\s*([^<]+)/i);
  const mRod = html.match(/Rodovia:\s*([^<]+)/i);
  const mAdm = html.match(/Administra[çc][aã]o:\s*([^<]+)/i);
  return {
    municipio: (mMun && mMun[1]) ? mMun[1].trim() : "—",
    rodovia:   (mRod && mRod[1]) ? mRod[1].trim() : "—",
    adm:       (mAdm && mAdm[1]) ? mAdm[1].trim() : "—",
  };
}
function coletarMarcosDoMapa(map) {
  const marcos = [];
  const layers = map && map._layers ? map._layers : {};
  for (const id in layers) {
    const l = layers[id];
    if (l instanceof L.Marker) {
      const latlng = l.getLatLng && l.getLatLng();
      const pop = (l.getPopup && l.getPopup()) ? l.getPopup().getContent() : "";
      if (!latlng || !pop || !/Rodovia:/i.test(pop)) continue;
      const info = extrairInfoPopup(pop);
      marcos.push({ lat: latlng.lat, lon: latlng.lng, municipio: info.municipio, rodovia: info.rodovia, concessionaria: info.adm });
    }
  }
  return marcos;
}
function carregarMarcosPreferindoJSON(map, cb) {
  fetch("marcos_geo_filtrados.json", { cache: "no-store" })
    .then(r => r.ok ? r.json() : Promise.reject(new Error("HTTP " + r.status)))
    .then(js => { if (Array.isArray(js) && js.length) cb(js); else cb(coletarMarcosDoMapa(map)); })
    .catch(() => cb(coletarMarcosDoMapa(map)));
}

let popupSaudacao = null, jaMostrou = false, estavaDentro = false;
function fecharSaudacao(map) { if (popupSaudacao) { map.closePopup(popupSaudacao); popupSaudacao = null; } }
function abrirOuAtualizarSaudacao(map, lat, lon, municipio, rodovia, adm) {
  const html = "<b>Bem-vindo à " + CIA_LABEL + "!</b><br>" +
               "Você está no município: " + (municipio || "—") + "<br>" +
               "Rodovia: " + (rodovia || "—") + "<br>" +
               "Administrada por: " + (adm || "—");
  if (!popupSaudacao) {
    popupSaudacao = L.popup({ closeOnClick: false, autoClose: false, className: "popup-saudacao" })
      .setLatLng([lat, lon]).setContent(html).openOn(map);
  } else {
    popupSaudacao.setLatLng([lat, lon]);
    if (popupSaudacao.getContent() !== html) popupSaudacao.setContent(html);
  }
}
function iniciarSaudacao(map) {
  if (!navigator.geolocation) { console.warn("Geolocalização não suportada."); return; }
  carregarMarcosPreferindoJSON(map, function(marcos) {
    navigator.geolocation.watchPosition(function(pos) {
      const lat = pos.coords.latitude, lon = pos.coords.longitude;
      const dentro = (typeof window.__MAPA_GEOFENCE_OK === "function") ? !!window.__MAPA_GEOFENCE_OK([lat, lon]) : true;
      if (!dentro) { fecharSaudacao(map); estavaDentro = false; if (MOSTRAR_APENAS_AO_ENTRAR) jaMostrou = false; return; }
      const entrouAgora = (!estavaDentro && dentro); estavaDentro = dentro;
      if (MOSTRAR_APENAS_AO_ENTRAR && jaMostrou && !entrouAgora) return;
      let maisProx = null, menor = Infinity;
      for (const m of marcos) { const d = calcularDistancia(lat, lon, m.lat, m.lon); if (d < menor) { menor = d; maisProx = m; } }
      let municipio = "—", rodovia = "—", adm = "—";
      if (maisProx && menor <= DISTANCIA_MAX_KM) { municipio = maisProx.municipio || "—"; rodovia = maisProx.rodovia || "—"; adm = maisProx.concessionaria || "—"; }
      abrirOuAtualizarSaudacao(map, lat, lon, municipio, rodovia, adm); jaMostrou = true;
    }, function(err) { console.error("Erro de geolocalização:", err); }, { enableHighAccuracy: true, maximumAge: 15000, timeout: 10000 });
  });
}
document.addEventListener("DOMContentLoaded", function() {
  const map = obterMapaFolium(); if (!map) return;
  if (location.protocol === "file:") console.warn("GPS pode ser bloqueado em file:// — use http://localhost");
  iniciarSaudacao(map);
});
"""
    mapa.get_root().html.add_child(Element("<script>\n" + SAUD_JS + "\n</script>"))

    # ---------- INJETAR DADOS (para o botão ℹ️ funcionar mesmo com cluster) ----------
    def _safe_int(v):
        try:
            return int(round(float(v)))
        except Exception:
            return None

    marcos_js = []
    for _, r in df_km.iterrows():
        marcos_js.append({
            "lat": float(r["y"]),
            "lon": float(r["x"]),
            "km": _safe_int(r["km"]),
            "municipio": (str(r["municipio"]) if pd.notna(r["municipio"]) else "—"),
            "sp": str(r["_ROD"]),
            "concessionaria": (str(r["concessionaria"]) if pd.notna(r["concessionaria"]) else "—")
        })
    mapa.get_root().html.add_child(Element("<script>\nwindow.__MARCO_DATA = " + json.dumps(marcos_js, ensure_ascii=False) + ";\n</script>"))

    # ---------- JS: botões 📍 e ℹ️ (agora usando whenReady + __MARCO_DATA) ----------
    mapa.get_root().html.add_child(Element(f"""
<script>
document.addEventListener('DOMContentLoaded', function() {{
  var map = (typeof obterMapaFolium === 'function') ? obterMapaFolium() : {map_var};
  if (!map || !map.whenReady) return;

  map.whenReady(function() {{
    // traduz / acessibilidade do botão de medir
    var btn = document.querySelector('.leaflet-control-measure a');
    if (btn) {{
      btn.setAttribute('title','Medir distância e áreas');
      btn.setAttribute('aria-label','Medir distância e áreas');
    }}

    function getMarcos() {{
      if (Array.isArray(window.__MARCO_DATA) && window.__MARCO_DATA.length) return window.__MARCO_DATA;
      return [];
    }}

    function mostrarInfoLocal() {{
      if(!navigator.geolocation) {{ alert('Geolocalização não suportada.'); return; }}
      navigator.geolocation.getCurrentPosition(function(pos){{
        var lat=pos.coords.latitude, lng=pos.coords.longitude;
        var dentro = (window.__MAPA_GEOFENCE_OK ? !!window.__MAPA_GEOFENCE_OK([lat,lng]) : true);
        var marcos = getMarcos();
        if(!marcos.length) {{ alert('Marcos quilométricos indisponíveis.'); return; }}
        var melhor=null, menor=Infinity;
        for(var i=0;i<marcos.length;i++) {{
          var m=marcos[i];
          var d=calcularDistancia(lat,lng,m.lat,m.lon);
          if(d<menor) {{ menor=d; melhor=m; }}
        }}
        var html;
        if (dentro) {{
          html = '<b>Local atual</b><br>'+
                 '<b>SP:</b> ' + (melhor && melhor.sp ? melhor.sp : '—') + '<br>'+
                 '<b>KM:</b> ' + (melhor && (melhor.km||melhor.km===0) ? melhor.km : '—') + '<br>'+
                 '<b>Município:</b> ' + (melhor ? (melhor.municipio||'—') : '—') + '<br>'+
                 '<b>Concessionária:</b> ' + (melhor ? (melhor.concessionaria||'—') : '—');
        }} else {{
          html = '<b>Fora da área da Companhia</b><br>'+
                 '<b>Rodovia:</b> ' + (melhor && melhor.sp ? melhor.sp : '—') + '<br>'+
                 '<b>KM:</b> ' + (melhor && (melhor.km||melhor.km===0) ? melhor.km : '—') + '<br>'+
                 '<b>Município:</b> ' + (melhor ? (melhor.municipio||'—') : '—');
        }}
        L.popup({{ closeOnClick:false, autoClose:false, className:'popup-infolocal' }})
          .setLatLng([lat,lng]).setContent(html).openOn(map);
      }}, function(err) {{
        alert('Não foi possível obter a localização. Verifique as permissões.');
      }}, {{enableHighAccuracy:true, maximumAge:15000, timeout:10000}});
    }}

    // Controle com dois botões (📍 e ℹ️)
    var CustomCtrl = L.Control.extend({{
      options: {{ position: 'topleft' }},
      onAdd: function(map) {{
        var div = L.DomUtil.create('div','leaflet-bar');
        // Botão 📍
        var a=L.DomUtil.create('a','',div);
        a.href='#'; a.title='Centralizar na minha posição'; a.innerHTML='📍';
        a.style.fontSize='18px'; a.style.lineHeight='26px'; a.style.textAlign='center'; a.style.width='26px'; a.style.height='26px';
        L.DomEvent.on(a,'click',function(e){{L.DomEvent.stop(e);
          if(!navigator.geolocation) return alert('Geolocalização não suportada.');
          navigator.geolocation.getCurrentPosition(function(pos){{
            var lat=pos.coords.latitude, lng=pos.coords.longitude;
            map.flyTo([lat,lng],15);
            L.circleMarker([lat,lng],{{radius:8,color:'#1976d2',fillColor:'#42a5f5',fillOpacity:0.8}}).addTo(map).bindPopup('Você está aqui').openPopup();
          }}, function(err){{ alert('Não foi possível obter a localização. Verifique as permissões.'); }},
          {{enableHighAccuracy:true, maximumAge:15000, timeout:10000}});
        }});

        // Botão ℹ️
        var info=L.DomUtil.create('a','',div);
        info.href='#'; info.title='Informações do local (SP/KM/Município e, se dentro da área, Concessionária)'; info.innerHTML='ℹ️';
        info.style.fontSize='18px'; info.style.lineHeight='26px'; info.style.textAlign='center'; info.style.width='26px'; info.style.height='26px';
        L.DomEvent.on(info,'click',function(e){{ L.DomEvent.stop(e); mostrarInfoLocal(); }});
        return div;
      }}
    }});
    map.addControl(new CustomCtrl());
  }});
}});
</script>
"""))

    # ---------- Salvar ----------
    mapa.save(str(SAIDA_HTML))
    print("✅ Mapa salvo em:", SAIDA_HTML)

if __name__ == "__main__":
    main()


✅ Mapa salvo em: mapa_completo_final.html


In [20]:
# -*- coding: utf-8 -*-
"""
Mapa completo — Folium/Leaflet (geolocalização + PT-BR no Medir + legenda à direita)
- Legenda no canto inferior direito (largura maior e até 70% da tela) com Mostrar/Ocultar.
- MiniMap no canto inferior esquerdo.
- Telefones das administradoras nos popups das LINHAS e também dos MARCOS KM.
- Saudação embutida ao ENTRAR na área da Cia (raio 1 km do marco mais próximo).
- Botões 📍 (centrar) e ℹ️ (popup de local).
  • DENTRO da área: SP/KM/Município/Concessionária a partir dos seus marcos.
  • FORA da área: Rodovia (ref/name), KM (se existir milestone), Município via Overpass/Nominatim.

Requisitos:
    pip install pandas folium shapely
Arquivos na mesma pasta:
    - Malha.xlsx
    - coordenadas.xlsx
    - municipios_selecionados.xlsx
    - bases.xlsx
Saída:
    - mapa_completo_final.html
"""

import json
import re
from pathlib import Path
from collections import defaultdict

import folium
from folium.plugins import (
    LocateControl, MarkerCluster, Fullscreen, MiniMap, MeasureControl
)
from folium.features import GeoJsonPopup
from shapely import wkt
import pandas as pd
import numpy as np
from branca.element import Element

# =========================
# Configurações de caminhos
# =========================
BASE = Path(".")
ARQ_KM = BASE / "coordenadas.xlsx"
ARQ_BASES = BASE / "bases.xlsx"
ARQ_MUN = BASE / "municipios_selecionados.xlsx"
ARQ_MALHA = BASE / "Malha.xlsx"
SAIDA_HTML = BASE / "mapa_completo_final.html"

# =========================
# Helpers de normalização
# =========================
def norm_rod_str(s: str) -> str:
    s = str(s).upper().strip()
    s = re.sub(r"\s+", " ", s)
    s = re.sub(r"^SPA\b", "SP", s)
    return s

def normalize_concessionaria(name: str) -> str:
    n = (name or "").strip().upper()
    if "EIXO" in n or "PIPA" in n: return "Concessionária EIXOSP - PIPA"
    if "ECOVIAS" in n: return "Concessionária Ecovias Noroeste Paulista"
    if "ENTREVIAS" in n: return "Concessionária Entrevias"
    if "INTERVIAS" in n: return "Concessionária Intervias"
    if "VIA PAULISTA" in n or "VIAPAULISTA" in n: return "Concessionária ViaPaulista"
    return f"Concessionária {name.strip()}" if name else "Concessionária"

def padroniza_admin_row(row) -> str:
    admin = str(row["ADMINISTRAÇÃO"]).upper().strip()
    cons = str(row["CONSERVAÇÃO"]).strip()
    sp_str = str(row["SP"]).strip()
    if "DER" in admin:
        m = re.search(r"(\d{2})", cons)
        if m: return f"DER {m.group(1)}"
        m2 = re.search(r"DER\D*(\d{2})", admin)
        if m2: return f"DER {m2.group(1)}"
        if sp_str in ("149/215", "348/310"): return "DER 04"
        return "DER 04"
    if "CONCESSION" in admin or "CONCESSIONÁRIA" in admin:
        return normalize_concessionaria(cons)
    if sp_str in ("149/215", "348/310"): return "DER 04"
    m = re.search(r"(\d{2})", cons)
    if m: return f"DER {m.group(1)}"
    return "DER 04"

def shapely_to_geojson_feature(r, cor, pelotao):
    popup_html = (
        f"<b>Município:</b> {r['name_muni']}<br>"
        f"<b>Pelotão:</b> {pelotao or '-'}<br>"
        f"<b>Batalhão:</b> {r.get('BTLH','-')}<br>"
        f"<b>CPI:</b> {r.get('CPI','-')}"
    )
    return {
        "type": "Feature",
        "properties": {"popup_html": popup_html},
        "geometry": r["geometry"].__geo_interface__,
    }

def geom_to_rings_latlng(geom):
    polys = []
    if geom.geom_type == "Polygon":
        ext = [[lat, lng] for (lng, lat) in list(geom.exterior.coords)]
        holes = [[[lat, lng] for (lng, lat) in list(inter.coords)] for inter in geom.interiors]
        polys.append([ext] + holes)
    elif geom.geom_type == "MultiPolygon":
        for p in geom.geoms:
            ext = [[lat, lng] for (lng, lat) in list(p.exterior.coords)]
            holes = [[[lat, lng] for (lng, lat) in list(inter.coords)] for inter in p.interiors]
            polys.append([ext] + holes)
    return polys

# =========================
# Pipeline principal
# =========================
def main():
    # ---------- Leitura ----------
    df_km = pd.read_excel(ARQ_KM)
    df_bases = pd.read_excel(ARQ_BASES)
    df_municipios = pd.read_excel(ARQ_MUN)
    df_malha = pd.read_excel(ARQ_MALHA, header=None, skiprows=1)

    df_malha.columns = [
        "Tipo","SP","KM INICIAL","KM FINAL","EXTENSÃO","MUNICÍPIO","RODOVIA","JURISDIÇÃO",
        "ADMINISTRAÇÃO","CONSERVAÇÃO","SUPERFÍCIE","PELOTÃO","Sentido","CPI","BTLH","CIA","PEL/GP"
    ]
    for c in ["KM INICIAL","KM FINAL"]:
        df_malha[c] = pd.to_numeric(df_malha[c], errors="coerce")

    # ---------- Normalizações ----------
    df_malha["ADMIN_FINAL"] = df_malha.apply(padroniza_admin_row, axis=1)
    df_malha["COD_ROD"] = (df_malha["Tipo"].astype(str) + " " + df_malha["SP"].astype(str)).apply(norm_rod_str)

    df_municipios["geometry"] = df_municipios["geometry"].apply(wkt.loads)

    info = (
        df_malha[["MUNICÍPIO","PELOTÃO","BTLH","CPI"]]
        .dropna(subset=["MUNICÍPIO"]).drop_duplicates("MUNICÍPIO")
    )
    mapeia_pel = {1:"1º Pel",2:"2º Pel",3:"3º Pel"}
    def map_pel(v):
        try:
            if pd.isna(v): return None
            return mapeia_pel.get(int(v))
        except Exception:
            return None
    info["PELOTÃO"] = info["PELOTÃO"].apply(map_pel)
    map_info = info.set_index("MUNICÍPIO").to_dict("index")

    df_municipios["PELOTÃO"] = df_municipios["name_muni"].map(lambda x: map_info.get(x,{}).get("PELOTÃO"))
    df_municipios["BTLH"] = df_municipios["name_muni"].map(lambda x: map_info.get(x,{}).get("BTLH"))
    df_municipios["CPI"] = df_municipios["name_muni"].map(lambda x: map_info.get(x,{}).get("CPI"))

    df_km["_ROD"] = df_km["rodovia"].apply(norm_rod_str)

    def get_info(row):
        mask = (
            (df_malha["COD_ROD"] == row["_ROD"]) &
            (df_malha["KM INICIAL"] <= float(row["km"])) &
            (df_malha["KM FINAL"] >= float(row["km"]))
        )
        t = df_malha.loc[mask]
        if not t.empty:
            t0 = t.iloc[0]
            return pd.Series([t0["MUNICÍPIO"], f"{t0['Tipo']} {t0['SP']} - {t0['RODOVIA']}", t0["ADMIN_FINAL"]])
        t2 = df_malha[df_malha["COD_ROD"] == row["_ROD"]]
        if not t2.empty:
            t0 = t2.iloc[0]
            return pd.Series([t0["MUNICÍPIO"], f"{t0['Tipo']} {t0['SP']} - {t0['RODOVIA']}", t0["ADMIN_FINAL"]])
        return pd.Series(["(indef.)", row["_ROD"], "DER 04"])

    df_km[["municipio","rodovia_nome","concessionaria"]] = df_km.apply(get_info, axis=1)

    # ---------- Telefones ----------
    phones = {
        "Concessionária EIXOSP - PIPA": "0800 170 8998",
        "Concessionária Ecovias Noroeste Paulista": "0800 326 3663",
        "Concessionária Entrevias": "0800 3000 333",
        "Concessionária Intervias": "0800 707 1414",
        "Concessionária ViaPaulista": "0800 001 1255",
        "DER 14": "0800 55 55 10",
        "DER 04": "0800 55 55 10",
    }
    def tel_pair(nome):
        tel = phones.get(str(nome), "-")
        digits = re.sub(r"\D+", "", tel) if tel != "-" else ""
        return tel, digits

    # ---------- Mapa base ----------
    mapa = folium.Map(location=[-21.5, -48.5], zoom_start=8,
                      control_scale=True, width="100%", height="100%")

    # Controles
    LocateControl(position="topleft", auto_start=False, keepCurrentZoomLevel=True, flyTo=True,
                  strings={"title":"Mostrar minha localização","popup":"Você está aqui","outsideMapBoundsMsg":"Você está fora da área do mapa"},
                  locateOptions={"enableHighAccuracy":True,"maximumAge":15000,"timeout":10000}).add_to(mapa)
    Fullscreen(position="topleft").add_to(mapa)
    MeasureControl(position="topleft",
                   primary_length_unit='kilometers', secondary_length_unit='meters',
                   primary_area_unit='hectares', secondary_area_unit='sqmeters').add_to(mapa)
    MiniMap(position="bottomleft", toggle_display=True).add_to(mapa)

    # ---------- Municípios ----------
    cores_pelotao = {"1º Pel":"blue","2º Pel":"green","3º Pel":"gray"}
    geojson_names = []
    for pelotao, sub in df_municipios.groupby("PELOTÃO"):
        fg = folium.FeatureGroup(name=f"{pelotao}", show=True)
        for _, r in sub.iterrows():
            cor = cores_pelotao.get(pelotao, "#CCCCCC")
            feat = shapely_to_geojson_feature(r, cor, pelotao)
            g = folium.GeoJson(
                data=feat, name=f"muni_{r['name_muni']}",
                style_function=lambda f, cor=cor: {"fillColor":cor,"color":"black","weight":1,"fillOpacity":0.35},
                highlight_function=lambda f: {"weight":3,"fillOpacity":0.6},
            )
            g.add_child(GeoJsonPopup(fields=["popup_html"], labels=False, parse_html=True, localize=True, max_width=320))
            g.add_to(fg); geojson_names.append(g.get_name())
        fg.add_to(mapa)

    # ---------- Marcos km (com telefone no popup) ----------
    counts = df_km.groupby(["x","y"]).size()
    delta = 0.00005
    offsets = {coord:list(np.linspace(-delta*(cnt-1)/2, delta*(cnt-1)/2, cnt))
               for coord, cnt in counts.items() if cnt > 1}
    used = defaultdict(int)
    cluster = MarkerCluster(name="Marcos Quilométricos").add_to(mapa)
    for _, r in df_km.iterrows():
        coord = (r["x"], r["y"])
        idx = used[coord]
        off = offsets.get(coord, [0])[idx] if coord in offsets else 0
        used[coord] += 1
        tel, tel_digits = tel_pair(r["concessionaria"])
        tel_html = f"<a href='tel:{tel_digits}'>{tel}</a>" if tel_digits else "-"
        folium.Marker(
            location=[r["y"] + off, r["x"]],
            icon=folium.DivIcon(html=f'<div class="km-label" style="font-size:9pt;font-weight:bold;color:black;">{int(r["km"])}</div>'),
            popup=folium.Popup(
                f"<b>Município:</b> {r['municipio']}<br>"
                f"<b>Rodovia:</b> {r['rodovia_nome']}<br>"
                f"<b>Administração:</b> {r['concessionaria']}<br>"
                f"<b>Telefone:</b> {tel_html}",
                max_width=340
            )
        ).add_to(cluster)

    # ---------- Linhas por Administração + telefone ----------
    cores_concessionarias = {
        "Concessionária EIXOSP - PIPA":"red",
        "Concessionária Ecovias Noroeste Paulista":"blue",
        "Concessionária Entrevias":"green",
        "Concessionária Intervias":"orange",
        "Concessionária ViaPaulista":"purple",
        "DER 14":"white", "DER 04":"gray",
    }
    for concess, grp in df_km.groupby("concessionaria"):
        fg = folium.FeatureGroup(name=f"Administração: {concess}", show=False)
        cor = cores_concessionarias.get(concess, "#666666")
        tel, tel_digits = tel_pair(concess)
        tel_html = f"<a href='tel:{tel_digits}'>{tel}</a>" if tel_digits else "-"
        for _, sub in grp.groupby("_ROD"):
            coords = sub.sort_values("km")[["y","x"]].values.tolist()
            folium.PolyLine(
                locations=coords, color=cor, weight=5, opacity=0.85,
                popup=folium.Popup(
                    f"<b>Administração:</b> {concess}<br>"
                    f"<b>Rodovia:</b> {sub['_ROD'].iloc[0]}<br>"
                    f"<b>Telefone:</b> {tel_html}",
                    max_width=340
                )
            ).add_to(fg)
        fg.add_to(mapa)

    # ---------- CSS + Legenda ----------
    style_css = """
<style>
#legend-mapa {
  position: fixed;
  bottom: 16px;
  right: 16px;
  width: 380px;
  max-height: 70vh;
  overflow: auto;
  background: white;
  z-index: 700;
  font-size: 14px;
  border: 2px solid #888;
  border-radius: 10px;
  box-shadow: 0 2px 12px rgba(0,0,0,.18);
  padding: 12px;
  line-height: 1.28;
}
#legend-mapa .legend-header{
  display:flex;align-items:center;justify-content:space-between;
  font-weight:700;margin-bottom:8px;
}
#legend-mapa button.legend-toggle{
  border:1px solid #bbb;border-radius:6px;background:#f6f6f6;
  padding:2px 8px;cursor:pointer;
}
#legend-mapa .sym{width:12px;height:12px;display:inline-block;margin-right:6px;border:1px solid #000;}
@media (max-width: 640px){
  #legend-mapa{ width: 300px; max-height: 60vh; }
}
#geoloc-warn{
  position:fixed;left:50%;transform:translateX(-50%);bottom:12px;
  background:#fff3cd;border:1px solid #ffeeba;padding:8px 12px;border-radius:8px;
  font-size:13px;z-index:1000;display:none;box-shadow:0 2px 8px rgba(0,0,0,.12);
}
</style>
"""
    mapa.get_root().html.add_child(Element(style_css))

    def tel_link(num: str) -> str:
        digits = re.sub(r"\D+", "", num or "")
        return f'<a href="tel:{digits}" style="text-decoration:none;color:#333;">{num}</a>' if digits else "-"

    legenda_body = '<b>Pelotões</b><br>'
    for nm, co in {"1º Pel":"blue","2º Pel":"green","3º Pel":"gray"}.items():
        legenda_body += f'<div style="margin:3px 0;"><i class="sym" style="background:{co};"></i> {nm}</div>'
    legenda_body += '<hr style="margin:8px 0;"><b>Concessionárias / DER</b><br>'
    for nm, co in cores_concessionarias.items():
        numero = phones.get(nm, "-")
        legenda_body += (
            f'<div style="margin:4px 0;">'
            f'<i class="sym" style="background:{co};"></i> {nm}<br>'
            f'<span style="margin-left:20px;font-size:12px;color:#333;">{tel_link(numero)}</span>'
            f'</div>'
        )

    legenda_html = f"""
<div id="legend-mapa">
  <div class="legend-header">
    Legenda
    <button class="legend-toggle"
      onclick="(function(btn){{var c=btn.closest('#legend-mapa');var b=c.querySelector('.legend-body');var sh=b.style.display!=='none';b.style.display=sh?'none':'block';btn.textContent=sh?'Mostrar':'Ocultar';}})(this)">Ocultar</button>
  </div>
  <div class="legend-body">
    {legenda_body}
  </div>
</div>
<div id="geoloc-warn">Para ativar geolocalização, abra o mapa via <b>http://localhost</b> (navegadores bloqueiam GPS em <code>file://</code>).</div>
"""
    mapa.get_root().html.add_child(Element(legenda_html))

    # ---------- Layer control ----------
    folium.LayerControl(collapsed=True).add_to(mapa)

    # ---------- JS: rótulos de KM apenas com zoom >= 12 ----------
    map_var = mapa.get_name()
    mapa.get_root().html.add_child(Element(f"""
<script>
(function() {{
  var map = {map_var};
  function toggleKmLabels() {{
    var show = map.getZoom() >= 12;
    document.querySelectorAll('.km-label').forEach(function(el) {{
      el.style.display = show ? 'block' : 'none';
    }});
  }}
  map.on('zoomend', toggleKmLabels);
  toggleKmLabels();
}})();
</script>
"""))

    # ---------- JS: Destaque e popup nos municípios ----------
    geo_list = ",".join(geojson_names)
    mapa.get_root().html.add_child(Element(f"""
<script>
(function() {{
   var map = {map_var};
   function baseStyle(layer) {{ return {{ color: layer.options.color||'black', weight: 1, fillOpacity: 0.35, fillColor: layer.options.fillColor||'#ccc' }}; }}
   function highlightStyle(layer) {{ var s=baseStyle(layer); s.weight=3; s.fillOpacity=0.6; return s; }}
   function reset(layer) {{ layer.setStyle(layer._baseStyle||baseStyle(layer)); }}
   function highlight(layer) {{ layer.setStyle(highlightStyle(layer)); }}
   function attach(layer) {{
      layer._baseStyle = baseStyle(layer);
      var handler = function(e) {{
         if (window._selectedMunicipio && window._selectedMunicipio !== layer) {{ reset(window._selectedMunicipio); }}
         highlight(layer); window._selectedMunicipio = layer;
         if (layer.getPopup && layer.getPopup()) {{ layer.openPopup(e && e.latlng ? e.latlng : undefined); }}
         if (e && e.originalEvent && e.originalEvent.preventDefault) e.originalEvent.preventDefault();
      }};
      layer.on('click', handler);
      layer.on('touchstart', handler);
      layer.on('mouseover', function() {{ if (window._selectedMunicipio !== layer) highlight(layer); }});
      layer.on('mouseout', function() {{ if (window._selectedMunicipio !== layer) reset(layer); }});
   }}
   var layers = [{geo_list}];
   layers.forEach(function(g) {{
      if (!g) return;
      g.eachLayer(function(l) {{
         if (l && l.eachLayer) {{
            l.eachLayer(function(poly) {{ if (poly instanceof L.Polygon || poly instanceof L.MultiPolygon) attach(poly); }});
         }} else if (l instanceof L.Polygon || l instanceof L.MultiPolygon) {{ attach(l); }}
      }});
   }});
   map.on('click', function() {{ if (window._selectedMunicipio) {{ reset(window._selectedMunicipio); window._selectedMunicipio=null; }} }});
}})();
</script>
"""))

    # ---------- JS: Geofencing + aviso file:// ----------
    geofence_polys = []
    for _, r in df_municipios.iterrows():
        geofence_polys.extend(geom_to_rings_latlng(r["geometry"]))
    mapa.get_root().html.add_child(Element(f"""
<script>
window.__MAPA_GEOFENCE = {json.dumps(geofence_polys)};
(function() {{ if (location.protocol === 'file:') {{ var el=document.getElementById('geoloc-warn'); if(el) el.style.display='block'; }} }})();
function __pip_in_ring(point, ring) {{
  var x = point[1], y = point[0], inside = false;
  for (var i=0, j=ring.length-1; i<ring.length; j=i++) {{
    var xi=ring[i][1], yi=ring[i][0], xj=ring[j][1], yj=ring[j][0];
    var intersect = ((yi>y)!=(yj>y)) && (x < (xj-xi)*(y-yi)/((yj-yi)||1e-12) + xi);
    if (intersect) inside = !inside;
  }}
  return inside;
}}
function __pip_in_poly(point, poly) {{
  if (!poly || !poly.length) return false;
  var exterior = poly[0];
  if (!__pip_in_ring(point, exterior)) return false;
  for (var h=1; h<poly.length; h++) if (__pip_in_ring(point, poly[h])) return false;
  return true;
}}
window.__MAPA_GEOFENCE_OK = function(latlng) {{
  if (!window.__MAPA_GEOFENCE || !window.__MAPA_GEOFENCE.length) return true;
  var p = [latlng[0], latlng[1]];
  for (var i=0;i<window.__MAPA_GEOFENCE.length;i++) if (__pip_in_poly(p, window.__MAPA_GEOFENCE[i])) return true;
  return false;
}};
</script>
"""))

    # ---------- JS: Saudação EMBUTIDA (raio 1 km) ----------
    SAUD_JS = r"""
// saudacao_localizacao.js — embutido (raio 1 km)
const CIA_LABEL = "1ª Cia do 3º BPRv";
const DISTANCIA_MAX_KM = 1.0;
const MOSTRAR_APENAS_AO_ENTRAR = true;

function calcularDistancia(lat1, lon1, lat2, lon2) {
  const R = 6371;
  const dLat = (lat2 - lat1) * Math.PI / 180;
  const dLon = (lon2 - lon1) * Math.PI / 180;
  const a = Math.sin(dLat/2)**2 +
            Math.cos(lat1*Math.PI/180) * Math.cos(lat2*Math.PI/180) *
            Math.sin(dLon/2)**2;
  const c = 2 * Math.atan2(Math.sqrt(a), Math.sqrt(1-a));
  return R * c;
}
function obterMapaFolium() {
  if (window.map && typeof window.map.setView === "function") return window.map;
  for (const k in window) {
    try { if (k.startsWith("map_") && window[k] && typeof window[k].setView === "function") return window[k]; } catch (_) {}
  }
  return null;
}
function extrairInfoPopup(html) {
  if (!html) return { municipio: "—", rodovia: "—", adm: "—" };
  const mMun = html.match(/Munic[ií]pio:\s*([^<]+)/i);
  const mRod = html.match(/Rodovia:\s*([^<]+)/i);
  const mAdm = html.match(/Administra[çc][aã]o:\s*([^<]+)/i);
  return {
    municipio: (mMun && mMun[1]) ? mMun[1].trim() : "—",
    rodovia:   (mRod && mRod[1]) ? mRod[1].trim() : "—",
    adm:       (mAdm && mAdm[1]) ? mAdm[1].trim() : "—",
  };
}
function coletarMarcosDoMapa(map) {
  const marcos = [];
  const layers = map && map._layers ? map._layers : {};
  for (const id in layers) {
    const l = layers[id];
    if (l instanceof L.Marker) {
      const latlng = l.getLatLng && l.getLatLng();
      const pop = (l.getPopup && l.getPopup()) ? l.getPopup().getContent() : "";
      if (!latlng || !pop || !/Rodovia:/i.test(pop)) continue;
      const info = extrairInfoPopup(pop);
      marcos.push({ lat: latlng.lat, lon: latlng.lng, municipio: info.municipio, rodovia: info.rodovia, concessionaria: info.adm });
    }
  }
  return marcos;
}
function carregarMarcosPreferindoJSON(map) {
  fetch("marcos_geo_filtrados.json", { cache: "no-store" })
    .then(r => r.ok ? r.json() : Promise.reject(new Error("HTTP " + r.status)))
    .then(js => { if (Array.isArray(js) && js.length) cb(js); else cb(coletarMarcosDoMapa(map)); })
    .catch(() => cb(coletarMarcosDoMapa(map)));
}

let popupSaudacao = null, jaMostrou = false, estavaDentro = false;
function fecharSaudacao(map) { if (popupSaudacao) { map.closePopup(popupSaudacao); popupSaudacao = null; } }
function abrirOuAtualizarSaudacao(map, lat, lon, municipio, rodovia, adm) {
  const html = "<b>Bem-vindo à " + CIA_LABEL + "!</b><br>" +
               "Você está no município: " + (municipio || "—") + "<br>" +
               "Rodovia: " + (rodovia || "—") + "<br>" +
               "Administrada por: " + (adm || "—");
  if (!popupSaudacao) {
    popupSaudacao = L.popup({ closeOnClick: false, autoClose: false, className: "popup-saudacao" })
      .setLatLng([lat, lon]).setContent(html).openOn(map);
  } else {
    popupSaudacao.setLatLng([lat, lon]);
    if (popupSaudacao.getContent() !== html) popupSaudacao.setContent(html);
  }
}
function iniciarSaudacao(map) {
  if (!navigator.geolocation) { console.warn("Geolocalização não suportada."); return; }
  carregarMarcosPreferindoJSON(map, function(marcos) {
    navigator.geolocation.watchPosition(function(pos) {
      const lat = pos.coords.latitude, lon = pos.coords.longitude;
      const dentro = (typeof window.__MAPA_GEOFENCE_OK === "function") ? !!window.__MAPA_GEOFENCE_OK([lat, lon]) : true;
      if (!dentro) { fecharSaudacao(map); estavaDentro = false; if (MOSTRAR_APENAS_AO_ENTRAR) jaMostrou = false; return; }
      const entrouAgora = (!estavaDentro && dentro); estavaDentro = dentro;
      if (MOSTRAR_APENAS_AO_ENTRAR && jaMostrou && !entrouAgora) return;
      let maisProx = null, menor = Infinity;
      for (const m of marcos) { const d = calcularDistancia(lat, lon, m.lat, m.lon); if (d < menor) { menor = d; maisProx = m; } }
      let municipio = "—", rodovia = "—", adm = "—";
      if (maisProx && menor <= DISTANCIA_MAX_KM) { municipio = maisProx.municipio || "—"; rodovia = maisProx.rodovia || "—"; adm = maisProx.concessionaria || "—"; }
      abrirOuAtualizarSaudacao(map, lat, lon, municipio, rodovia, adm); jaMostrou = true;
    }, function(err) { console.error("Erro de geolocalização:", err); }, { enableHighAccuracy: true, maximumAge: 15000, timeout: 10000 });
  });
}
document.addEventListener("DOMContentLoaded", function() {
  const map = obterMapaFolium(); if (!map) return;
  if (location.protocol === "file:") console.warn("GPS pode ser bloqueado em file:// — use http://localhost");
  iniciarSaudacao(map);
});
"""
    mapa.get_root().html.add_child(Element("<script>\n" + SAUD_JS + "\n</script>"))

    # ---------- INJETAR DADOS MARCOS ----------
    def _safe_int(v):
        try:
            return int(round(float(v)))
        except Exception:
            return None
    marcos_js = []
    for _, r in df_km.iterrows():
        marcos_js.append({
            "lat": float(r["y"]),
            "lon": float(r["x"]),
            "km": _safe_int(r["km"]),
            "municipio": (str(r["municipio"]) if pd.notna(r["municipio"]) else "—"),
            "sp": str(r["_ROD"]),
            "concessionaria": (str(r["concessionaria"]) if pd.notna(r["concessionaria"]) else "—")
        })
    mapa.get_root().html.add_child(Element("<script>\nwindow.__MARCO_DATA = " + json.dumps(marcos_js, ensure_ascii=False) + ";\n</script>"))

    # ---------- JS: botões 📍 e ℹ️ (com fallback fora da área) ----------
    mapa.get_root().html.add_child(Element(f"""
<script>
document.addEventListener('DOMContentLoaded', function() {{
  var map = (typeof obterMapaFolium === 'function') ? obterMapaFolium() : {map_var};
  if (!map || !map.whenReady) return;

  map.whenReady(function() {{

    // Configurações do fallback online
    const ONLINE_FALLBACK = true;
    const OVERPASS_URL = "https://overpass-api.de/api/interpreter";
    const NOMINATIM_BASE = "https://nominatim.openstreetmap.org/reverse?format=jsonv2&accept-language=pt-BR";

    function overpass(query) {{
      return fetch(OVERPASS_URL, {{
        method: 'POST',
        headers: {{'Content-Type':'application/x-www-form-urlencoded; charset=UTF-8'}},
        body: 'data=' + encodeURIComponent(query)
      }}).then(r => {{
        if(!r.ok) throw new Error('Overpass HTTP '+r.status);
        return r.json();
      }});
    }}

    function reverseGeocode(lat, lon) {{
      const url = NOMINATIM_BASE + "&lat=" + lat + "&lon=" + lon;
      return fetch(url, {{method:'GET'}}).then(r => {{
        if(!r.ok) throw new Error('Nominatim HTTP '+r.status);
        return r.json();
      }}).then(js => {{
        const a = js.address || {{}};
        return a.city || a.town || a.village || a.municipality || a.county || "—";
      }});
    }}

    function calcularDistancia(lat1, lon1, lat2, lon2) {{
      const R = 6371;
      const dLat = (lat2 - lat1) * Math.PI / 180;
      const dLon = (lon2 - lon1) * Math.PI / 180;
      const a = Math.sin(dLat/2)**2 + Math.cos(lat1*Math.PI/180) * Math.cos(lat2*Math.PI/180) * Math.sin(dLon/2)**2;
      const c = 2 * Math.atan2(Math.sqrt(a), Math.sqrt(1-a));
      return R * c;
    }}

    function getMarcos() {{
      if (Array.isArray(window.__MARCO_DATA) && window.__MARCO_DATA.length) return window.__MARCO_DATA;
      return [];
    }}

    // ========= FIX: construir as queries por concatenação (sem {{lat}}/{{lon}}) =========
    async function buscarRodoviaMaisProxima(lat, lon) {{
      const q =
        '[out:json][timeout:15];(' +
        'way(around:4000,' + lat + ',' + lon + ')[\"highway\"][\"ref\"];' +
        'way(around:4000,' + lat + ',' + lon + ')[\"highway\"][\"name\"];' +
        ');out tags geom;';
      const data = await overpass(q);
      if (!data.elements || !data.elements.length) return {{ ref: "—", name: "—", nearestDist: Infinity }};
      let best = null, bestD = Infinity;
      for (const el of data.elements) {{
        const geom = el.geometry || [];
        for (let i=0;i<geom.length;i++) {{
          const d = calcularDistancia(lat, lon, geom[i].lat, geom[i].lon);
          if (d < bestD) {{ bestD = d; best = el; }}
        }}
      }}
      return {{
        ref: (best && best.tags && (best.tags.ref || best.tags["ref:br"] || best.tags.name)) || "—",
        name: (best && best.tags && best.tags.name) || "—",
        nearestDist: bestD
      }};
    }}

    async function buscarMarcoKM(lat, lon) {{
      const q =
        '[out:json][timeout:15];' +
        'node(around:10000,' + lat + ',' + lon + ')[\"highway\"=\"milestone\"];' +
        'out tags center;';
      const data = await overpass(q).catch(() => null);
      if (!data || !data.elements || !data.elements.length) return null;
      let best = null, bestD = Infinity;
      for (const el of data.elements) {{
        const nlat = el.lat || (el.center && el.center.lat), nlon = el.lon || (el.center && el.center.lon);
        if (nlat == null || nlon == null) continue;
        const d = calcularDistancia(lat, lon, nlat, nlon);
        const distTag = el.tags && (el.tags.distance || el.tags.km || el.tags.milestone);
        const m = distTag ? String(distTag).match(/(\\d+(?:[\\.,]\\d+)?)/) : null;
        const km = m ? m[1].replace(',', '.') : null;
        if (d < bestD && km) {{ bestD = d; best = {{km: km, d: d}}; }}
      }}
      return best; // {{km, d}} ou null
    }}
    // =============================================================================

    function montarPopupDentro(melhor) {{
      return '<b>Local atual</b><br>'+
             '<b>SP:</b> ' + (melhor && melhor.sp ? melhor.sp : '—') + '<br>'+
             '<b>KM:</b> ' + (melhor && (melhor.km||melhor.km===0) ? melhor.km : '—') + '<br>'+
             '<b>Município:</b> ' + (melhor ? (melhor.municipio||'—') : '—') + '<br>'+
             '<b>Concessionária:</b> ' + (melhor ? (melhor.concessionaria||'—') : '—');
    }}

    function montarPopupFora(rodoviaRef, rodoviaName, kmStr, municipio) {{
      let rod = (rodoviaRef && rodoviaRef !== '—') ? rodoviaRef : rodoviaName || '—';
      return '<b>Fora da área da Companhia</b><br>'+
             '<b>Rodovia:</b> ' + (rod || '—') + '<br>'+
             '<b>KM:</b> ' + (kmStr || '—') + '<br>'+
             '<b>Município:</b> ' + (municipio || '—');
    }}

    function mostrarDentro(lat, lon) {{
      const marcos = getMarcos();
      if(!marcos.length) {{
        L.popup({{ closeOnClick:false, autoClose:false }}).setLatLng([lat,lon]).setContent('Marcos indisponíveis.').openOn(map);
        return;
      }}
      let melhor=null, menor=Infinity;
      for (const m of marcos) {{
        const d=calcularDistancia(lat,lon,m.lat,m.lon);
        if(d<menor) {{ menor=d; melhor=m; }}
      }}
      L.popup({{ closeOnClick:false, autoClose:false, className:'popup-infolocal' }})
        .setLatLng([lat,lon]).setContent(montarPopupDentro(melhor)).openOn(map);
    }}

    async function mostrarFora(lat, lon) {{
      if (!ONLINE_FALLBACK || !navigator.onLine) {{
        L.popup({{ closeOnClick:false, autoClose:false }}).setLatLng([lat,lon])
          .setContent('Sem fallback online: conecte-se à internet.').openOn(map);
        return;
      }}
      try {{
        const [rod, muni, marco] = await Promise.all([
          buscarRodoviaMaisProxima(lat, lon),
          reverseGeocode(lat, lon),
          buscarMarcoKM(lat, lon)
        ]);
        const kmStr = marco ? ('' + marco.km) : '—';
        const html = montarPopupFora(rod.ref, rod.name, kmStr, muni);
        L.popup({{ closeOnClick:false, autoClose:false, className:'popup-infolocal' }})
          .setLatLng([lat,lon]).setContent(html).openOn(map);
      }} catch(e) {{
        L.popup({{ closeOnClick:false, autoClose:false }}).setLatLng([lat,lon])
          .setContent('Falha no fallback online. Tente novamente.').openOn(map);
        console.error(e);
      }}
    }}

    function onInfoClick() {{
      if(!navigator.geolocation) {{ alert('Geolocalização não suportada.'); return; }}
      navigator.geolocation.getCurrentPosition(function(pos){{
        const lat=pos.coords.latitude, lon=pos.coords.longitude;
        const dentro = (window.__MAPA_GEOFENCE_OK ? !!window.__MAPA_GEOFENCE_OK([lat,lon]) : true);
        if (dentro) mostrarDentro(lat, lon);
        else        mostrarFora(lat, lon);
      }}, function(err) {{
        alert('Não foi possível obter a localização. Verifique as permissões.');
      }}, {{enableHighAccuracy:true, maximumAge:15000, timeout:10000}});
    }}

    // Controle com dois botões (📍 e ℹ️)
    var CustomCtrl = L.Control.extend({{
      options: {{ position: 'topleft' }},
      onAdd: function(map) {{
        var div = L.DomUtil.create('div','leaflet-bar');

        // Botão 📍
        var a=L.DomUtil.create('a','',div);
        a.href='#'; a.title='Centralizar na minha posição'; a.innerHTML='📍';
        a.style.fontSize='18px'; a.style.lineHeight='26px'; a.style.textAlign='center'; a.style.width='26px'; a.style.height='26px';
        L.DomEvent.on(a,'click',function(e){{L.DomEvent.stop(e);
          if(!navigator.geolocation) return alert('Geolocalização não suportada.');
          navigator.geolocation.getCurrentPosition(function(pos){{
            var lat=pos.coords.latitude, lng=pos.coords.longitude;
            map.flyTo([lat,lng],15);
            L.circleMarker([lat,lng],{{radius:8,color:'#1976d2',fillColor:'#42a5f5',fillOpacity:0.8}}).addTo(map).bindPopup('Você está aqui').openPopup();
          }}, function(err){{ alert('Não foi possível obter a localização. Verifique as permissões.'); }},
          {{enableHighAccuracy:true, maximumAge:15000, timeout:10000}});
        }});

        // Botão ℹ️
        var info=L.DomUtil.create('a','',div);
        info.href='#'; info.title='Informações do local'; info.innerHTML='ℹ️';
        info.style.fontSize='18px'; info.style.lineHeight='26px'; info.style.textAlign='center'; info.style.width='26px'; info.style.height='26px';
        L.DomEvent.on(info,'click',function(e){{ L.DomEvent.stop(e); onInfoClick(); }});
        return div;
      }}
    }});
    map.addControl(new CustomCtrl());

  }}); // whenReady
}}); // DOMContentLoaded
</script>
"""))

    # ---------- Salvar ----------
    mapa.save(str(SAIDA_HTML))
    print("✅ Mapa salvo em:", SAIDA_HTML)

if __name__ == "__main__":
    main()


✅ Mapa salvo em: mapa_completo_final.html
